In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import GradientBoostingClassifier, HistGradientBoostingClassifier #Second is for data size >10k. CLASSIFICATION.
from sklearn.ensemble import GradientBoostingRegressor, HistGradientBoostingRegressor #as above. REGRESSION.
# import lightgbm

In [23]:
df = pd.read_csv('train_top_20k.csv')
df = df[df.columns.tolist()[1:]] #Drops the duplicated index values in the first column.
df.dropna(subset=['target'], inplace=True)
df.head()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.0,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.0,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.0,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.0,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.0,-7.349849,0,0_0_4


#### Some basic notes on the column definitions, units, etc:
- imbalance_size is defined as the "amount unmatched at the current reference price (in USD)"
- imbalance_buy_sell_flag gives the DIRECTION of the auction imbalance
- reference_price is the price at which paired shares are maximized, the imbalance is minimized and the distance from the bid-ask midpoint is minimized (in that order). **What does this mean exactly?**

In [24]:
weight_arr = [
    0.004, 0.001, 0.002, 0.006, 0.004, 0.004, 0.002, 0.006, 0.006, 0.002, 0.002, 0.008,
    0.006, 0.002, 0.008, 0.006, 0.002, 0.006, 0.004, 0.002, 0.004, 0.001, 0.006, 0.004,
    0.002, 0.002, 0.004, 0.002, 0.004, 0.004, 0.001, 0.001, 0.002, 0.002, 0.006, 0.004,
    0.004, 0.004, 0.006, 0.002, 0.002, 0.04 , 0.002, 0.002, 0.004, 0.04 , 0.002, 0.001,
    0.006, 0.004, 0.004, 0.006, 0.001, 0.004, 0.004, 0.002, 0.006, 0.004, 0.006, 0.004,
    0.006, 0.004, 0.002, 0.001, 0.002, 0.004, 0.002, 0.008, 0.004, 0.004, 0.002, 0.004,
    0.006, 0.002, 0.004, 0.004, 0.002, 0.004, 0.004, 0.004, 0.001, 0.002, 0.002, 0.008,
    0.02 , 0.004, 0.006, 0.002, 0.02 , 0.002, 0.002, 0.006, 0.004, 0.002, 0.001, 0.02,
    0.006, 0.001, 0.002, 0.004, 0.001, 0.002, 0.006, 0.006, 0.004, 0.006, 0.001, 0.002,
    0.004, 0.006, 0.006, 0.001, 0.04 , 0.006, 0.002, 0.004, 0.002, 0.002, 0.006, 0.002,
    0.002, 0.004, 0.006, 0.006, 0.002, 0.002, 0.008, 0.006, 0.004, 0.002, 0.006, 0.002,
    0.004, 0.006, 0.002, 0.004, 0.001, 0.004, 0.002, 0.004, 0.008, 0.006, 0.008, 0.002,
    0.004, 0.002, 0.001, 0.004, 0.004, 0.004, 0.006, 0.008, 0.004, 0.001, 0.001, 0.002,
    0.006, 0.004, 0.001, 0.002, 0.006, 0.004, 0.006, 0.008, 0.002, 0.002, 0.004, 0.002,
    0.04 , 0.002, 0.002, 0.004, 0.002, 0.002, 0.006, 0.02 , 0.004, 0.002, 0.006, 0.02,
    0.001, 0.002, 0.006, 0.004, 0.006, 0.004, 0.004, 0.004, 0.004, 0.002, 0.004, 0.04,
    0.002, 0.008, 0.002, 0.004, 0.001, 0.004, 0.006, 0.004,
]
stock_id_list = np.arange(200)
weights_df = pd.DataFrame({'stock_id':stock_id_list, 'weight':weight_arr})
df = df.merge(weights_df, how='left', on='stock_id')
df.head()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id,weight
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.0,-3.029704,0,0_0_0,0.004
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.0,-5.519986,0,0_0_1,0.001
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.0,-8.389950,0,0_0_2,0.002
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.0,-4.010200,0,0_0_3,0.006
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.0,-7.349849,0,0_0_4,0.004


In [25]:
#Cell to explore the above loaded df if needed without disturbing what is below.

# df[~df['far_price'].isna()]

# test = df.groupby(['stock_id']).rolling(3)['bid_price'].mean().fillna(method='bfill')
# df['ewmas_3'] = np.zeros(len(df))
# df['ewmas_3'] = np.where(df['stock_id']==0, test, df['ewmas_3'])
# print(df.groupby('stock_id').rolling(1)['bid_price'].mean()[0].head(20))
# test.head(20)

In [26]:
# df[df['stock_id']==0].head(20)

#### Next cell will be adding some BASIC features to the data as manual (domain-specific) augmentation. 
TBD: how to incorporate formal kernel methods if that is of interest.

In [27]:
import warnings
warnings.filterwarnings('ignore')

#Add basics like categorical target (for classification tree), book skew + other simple data transforms.
df['book_skew'] = np.log(df['bid_size']/df['ask_size'])
df['target_cat'] = np.where(df['target'] < 0, -1, np.where(df['target'] > 0, 1, 0)) #should we make this binary?

#Add some moving-average style transformations of important variables.
ewma_windows = [3,5,10]
ewma_cols = ['imbalance_size', 'book_skew', 'bid_price', 'ask_price']
for w in ewma_windows:
    for col in ewma_cols:
        df[f'ewmas_{col}_{w}'] = np.zeros(len(df))
        for id in df['stock_id'].unique():
            ma = df.groupby(['stock_id']).rolling(w)[col].mean().fillna(method='bfill').loc[id] #This is a little sketchy, maybe worth changing?
            # df[f'ewmas_{col}_{w}'] = np.where(df['stock_id']==id, ma, df[f'ewmas_{col}_{w}'])
            df[f'ewmas_{col}_{w}'][ma.index] = ma
        # print(f'ewmas_{col}_{w}', np.sum(df[f'ewmas_{col}_{w}']==0))



# df[['book_skew', 'ewmas_book_skew_5', 'bid_price', 'ask_price', 'ewmas_bid_price_5', 'ewmas_ask_price_5', "stock_id", 'date_id']].head(20)

ewmas_imbalance_size_3 2197
ewmas_book_skew_3 0
ewmas_bid_price_3 0
ewmas_ask_price_3 0
ewmas_imbalance_size_5 1945
ewmas_book_skew_5 0
ewmas_bid_price_5 0
ewmas_ask_price_5 0
ewmas_imbalance_size_10 1517
ewmas_book_skew_10 0
ewmas_bid_price_10 0
ewmas_ask_price_10 0


In [28]:
# test = df[df['ewmas_imbalance_size_3']== 0][['stock_id', 'date_id', 'imbalance_size', 'ewmas_imbalance_size_3']]
# test['stock_id'].unique()

In [29]:
#Derive the train/test params for our toy model.
print(df.time_id.unique()) #gonna use up to 40 for training and the rest for testing

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52]


In [42]:
#Define the train-test split.
X_train = df[df['time_id'] <=40].drop(columns=['target', 'target_cat'])
X_train = X_train.fillna(1e-9) #For imputing NaN values. Does this value make a big difference to performance?
X_test = df[df['time_id'] >40].drop(columns=['target', 'target_cat'])
X_test = X_test.fillna(1e-9) #See above.
print("Xtrain length", len(X_train)); print("Xtest length", len(X_test))
y_train = df[df['time_id'] <=40]['target']
y_test = df[df['time_id'] >40]['target']

#Filter for final feature set.
features = [c for c in df.columns if not c in ['stock_id','date_id','seconds_in_bucket','target_cat', 'target']]
print("Number of features used: ", len(features))
X_train = X_train[features]
X_test = X_test[features]

X_train.head()

Xtrain length 7831
Xtest length 2169
Number of features used:  27


,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,...,ewmas_bid_price_3,ewmas_ask_price_3,ewmas_imbalance_size_5,ewmas_book_skew_5,ewmas_bid_price_5,ewmas_ask_price_5,ewmas_imbalance_size_10,ewmas_book_skew_10,ewmas_bid_price_10,ewmas_ask_price_10
0,3180602.69,1,0.999812,13380276.64,1.000000e-09,1.000000e-09,0.999812,60651.50,1.000026,8493.03,...,0.999812,0.999990,1.659625e+06,0.017416,0.999941,1.000112,1.437813e+06,0.020892,1.000134,1.000283
1,166603.91,-1,0.999896,1642214.25,1.000000e-09,1.000000e-09,0.999896,3233.04,1.000660,20605.09,...,0.999896,1.000660,1.653446e+05,-0.858154,0.999896,1.000519,1.587247e+05,-0.806547,0.999826,1.000325
2,302879.87,-1,0.999561,1819368.03,1.000000e-09,1.000000e-09,0.999403,37956.00,1.000298,18995.00,...,0.999333,1.000158,1.993880e+05,2.490127,0.999277,1.000056,1.845573e+05,1.395419,0.999298,1.000088
3,11917682.27,-1,1.000171,18389745.62,1.000000e-09,1.000000e-09,0.999999,2324.90,1.000214,479032.40,...,0.999956,1.000128,1.115098e+07,-0.573014,0.999956,1.000154,1.110936e+07,0.087768,0.999995,1.000193
4,447549.96,-1,0.999532,17860614.95,1.000000e-09,1.000000e-09,0.999394,16485.54,1.000016,434.10,...,0.999809,1.000200,4.318647e+05,1.203269,0.999823,1.000154,4.393167e+05,-0.269244,0.999850,1.000133


In [40]:
#Fit classifier.
clf = GradientBoostingRegressor(loss='absolute_error', learning_rate=0.05, n_estimators=1000, verbose=1, random_state=42)
clf.fit(X_train, y_train)

      Iter       Train Loss   Remaining Time 
         1           5.7755           53.79s
         2           5.7572           52.58s
         3           5.7415           52.01s
         4           5.7257           51.57s
         5           5.7120           51.56s
         6           5.7002           51.44s
         7           5.6881           51.28s
         8           5.6770           51.19s
         9           5.6661           51.23s
        10           5.6573           51.26s
        20           5.5868           50.77s
        30           5.5338           50.17s
        40           5.4763           49.48s
        50           5.4316           48.92s
        60           5.3853           48.30s
        70           5.3456           47.76s
        80           5.3149           47.22s
        90           5.2822           46.74s
       100           5.2532           46.17s
       200           5.0529           41.04s
       300           4.8808           35.91s
       40

GradientBoostingRegressor(learning_rate=0.05, loss='absolute_error',
                          n_estimators=1000, random_state=42, verbose=1)

In [43]:
#Run prediction.
y_pred = clf.predict(X_test)
prediction_mae = np.linalg.norm(y_pred-y_test,ord=1)/len(y_pred)
print(prediction_mae)

4.501900024182004


In [44]:
#Fits a classifier with early stopping and min impurity criterion for limiting overfitting.
#Should we include cost-complexity pruning as well?
clf2 = GradientBoostingRegressor(loss='absolute_error', learning_rate=0.05, n_estimators=1000, verbose=1, random_state=42,
                                validation_fraction=0.2, n_iter_no_change=3, min_impurity_decrease=0.1)
clf2.fit(X_train, y_train)

      Iter       Train Loss   Remaining Time 
         1           5.8123           47.24s
         2           5.7932           44.44s
         3           5.7768           43.27s
         4           5.7613           42.53s
         5           5.7487           42.12s
         6           5.7360           41.99s
         7           5.7238           41.75s
         8           5.7126           41.53s
         9           5.7030           41.34s
        10           5.6940           41.18s
        20           5.6186           40.31s
        30           5.5580           39.72s
        40           5.5149           39.24s
        50           5.4632           38.84s
        60           5.4190           38.39s
        70           5.3715           37.98s
        80           5.3306           37.60s
        90           5.2854           37.21s
       100           5.2511           36.81s


GradientBoostingRegressor(learning_rate=0.05, loss='absolute_error',
                          min_impurity_decrease=0.1, n_estimators=1000,
                          n_iter_no_change=3, random_state=42,
                          validation_fraction=0.2, verbose=1)

In [45]:
#Prediction using the limited classifier.
y_pred2 = clf2.predict(X_test)
prediction2_mae = np.linalg.norm(y_pred2-y_test,ord=1)/len(y_pred2)
print(prediction2_mae)

4.5166027911845665


In [46]:
clf2.n_estimators_

145